# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [87]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [88]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [89]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': '/EAMNGTSRE SECB S(AKA)LDGOQTA LLN', 'calculationPrice': 'close', 'open': 127.62, 'openTime': 1655210264644, 'openSource': 'fifacloi', 'close': 133.7, 'closeTime': 1627515706472, 'closeSource': 'fficlioa', 'high': 131.27, 'highTime': 1662894481271, 'highSource': 'e eut5iei lye1a mdrpcnd', 'low': 132.17, 'lowTime': 1676120554503, 'lowSource': 'uacle ytdme 5n 1prdieei', 'latestPrice': 129.71, 'latestSource': 'Close', 'latestTime': 'December 16, 2020', 'latestUpdate': 1686819283044, 'latestVolume': 101676834, 'iexRealtimePrice': 129.015, 'iexRealtimeSize': 3, 'iexLastUpdated': 1633818548486, 'delayedPrice': 131.336, 'delayedPriceTime': 1679819138291, 'oddLotDelayedPrice': 129.65, 'oddLotDelayedPriceTime': 1665363416448, 'extendedPrice': 130.73, 'extendedChange': -0.06, 'extendedChangePercent': -0.00048, 'extendedPriceTime': 1624484195914, 'previousClose': 134.18, 'previousVolume': 159366662, 'change': -0.07, 'changePercent'

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [90]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

39.37

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [91]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [92]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,121.88,55.21,N/A
1,AAL,17.45,-1.24,N/A
2,AAP,162.75,24.39,N/A
3,AAPL,130.58,39.69,N/A
4,ABBV,109.62,23.14,N/A
...,...,...,...,...
500,YUM,108.46,33.68,N/A
501,ZBH,153.56,-209.09,N/A
502,ZBRA,392.90,43.68,N/A
503,ZION,43.22,18.24,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [93]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True, drop = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,33.900,2,N/A
1,NLOK,21.260,4.02,N/A
2,AIV,4.650,4.5,N/A
3,UNM,22.430,4.88,N/A
4,BIO,582.090,5.15,N/A
5,KIM,15.480,6.7,N/A
6,ALL,106.990,7.62,N/A
7,MET,46.940,7.81,N/A
8,CPB,48.700,8.2,N/A
9,BIIB,252.610,8.2,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [94]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [95]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [96]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,33.900,2,5899
1,NLOK,21.260,4.02,9407
2,AIV,4.650,4.5,43010
3,UNM,22.430,4.88,8916
4,BIO,582.090,5.15,343
5,KIM,15.480,6.7,12919
6,ALL,106.990,7.62,1869
7,MET,46.940,7.81,4260
8,CPB,48.700,8.2,4106
9,BIIB,252.610,8.2,791


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [97]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data[symbol]['advanced-stats'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']


# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


print(ev_to_gross_profit)

21.01404576531153


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [98]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
                        pd.Series([
                            symbol,
                            data[symbol]['quote']['latestPrice'],
                            'N/A',
                            data[symbol]['quote']['peRatio'],
                            'N/A',
                            data[symbol]['advanced-stats']['priceToBook'],
                            'N/A',
                            data[symbol]['advanced-stats']['priceToSales'],
                            'N/A',
                            ev_to_ebitda,
                            'N/A',
                            ev_to_gross_profit,
                            'N/A',
                            'N/A'
                        ],
                        index = rv_columns),
                            ignore_index = True
                        )
        

In [99]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,122.26,N/A,55.63,N/A,7.61,N/A,7.24,N/A,34.443227,N/A,13.674757,N/A,N/A
1,AAL,17.20,N/A,-1.24,N/A,-1.86,N/A,0.4187,N/A,-6.221289,N/A,1.417907,N/A,N/A
2,AAP,166.80,N/A,24.27,N/A,3,N/A,1.11,N/A,11.203493,N/A,2.508853,N/A,N/A
3,AAPL,128.30,N/A,40.57,N/A,34.89,N/A,7.95,N/A,27.462022,N/A,20.229104,N/A,N/A
4,ABBV,106.25,N/A,23.89,N/A,12.6,N/A,4.67,N/A,15.968442,N/A,9.340323,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.11,N/A,33.74,N/A,-4,N/A,6.1,N/A,24.008699,N/A,10.403930,N/A,N/A
501,ZBH,149.86,N/A,-204.78,N/A,2.7,N/A,4.5,N/A,26.457060,N/A,7.714812,N/A,N/A
502,ZBRA,389.14,N/A,45.09,N/A,10.62,N/A,4.73,N/A,29.473261,N/A,11.157335,N/A,N/A
503,ZION,43.89,N/A,17.93,N/A,0.9909,N/A,2.29,N/A,10.189365,N/A,2.600916,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [100]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,209.450,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,222.180,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.13,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,25.830,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.600,N/A,14.96,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,54.500,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.500,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,141.180,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,10.010,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1782.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [101]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [102]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [103]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
        
        
# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe       

0      0.865347
1      0.140594
2      0.463366
3      0.744554
4      0.453465
         ...   
500    0.671287
501    0.019802
502    0.780198
503    0.347525
504     0.80396
Name: PE Percentile, Length: 505, dtype: object
0       0.728713
1      0.0455446
2       0.418812
3       0.960396
4       0.855446
         ...    
500    0.0415842
501     0.381188
502     0.829703
503    0.0970297
504     0.942574
Name: PB Percentile, Length: 505, dtype: object
0       0.817822
1      0.0316832
2       0.171287
3       0.843564
4       0.676238
         ...    
500     0.767327
501     0.618812
502     0.682178
503     0.385149
504     0.930693
Name: PS Percentile, Length: 505, dtype: object
0       0.851485
1      0.0277228
2       0.277228
3       0.792079
4       0.493069
         ...    
500     0.742574
501     0.782178
502     0.825743
503     0.225743
504     0.859406
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.823762
1      0.0554455
2       0.128713
3       0.942

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,122.26,N/A,55.63,0.865347,7.6100,0.728713,7.2400,0.817822,34.443227,0.851485,13.674757,0.823762,N/A
1,AAL,17.20,N/A,-1.24,0.140594,-1.8600,0.0455446,0.4187,0.0316832,-6.221289,0.0277228,1.417907,0.0554455,N/A
2,AAP,166.80,N/A,24.27,0.463366,3.0000,0.418812,1.1100,0.171287,11.203493,0.277228,2.508853,0.128713,N/A
3,AAPL,128.30,N/A,40.57,0.744554,34.8900,0.960396,7.9500,0.843564,27.462022,0.792079,20.229104,0.942574,N/A
4,ABBV,106.25,N/A,23.89,0.453465,12.6000,0.855446,4.6700,0.676238,15.968442,0.493069,9.340323,0.645545,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.11,N/A,33.74,0.671287,-4.0000,0.0415842,6.1000,0.767327,24.008699,0.742574,10.403930,0.69505,N/A
501,ZBH,149.86,N/A,-204.78,0.019802,2.7000,0.381188,4.5000,0.618812,26.457060,0.782178,7.714812,0.49703,N/A
502,ZBRA,389.14,N/A,45.09,0.780198,10.6200,0.829703,4.7300,0.682178,29.473261,0.825743,11.157335,0.712871,N/A
503,ZION,43.89,N/A,17.93,0.347525,0.9909,0.0970297,2.2900,0.385149,10.189365,0.225743,2.600916,0.136634,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [104]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,122.26,N/A,55.63,0.865347,7.6100,0.728713,7.2400,0.817822,34.443227,0.851485,13.674757,0.823762,0.817426
1,AAL,17.20,N/A,-1.24,0.140594,-1.8600,0.0455446,0.4187,0.0316832,-6.221289,0.0277228,1.417907,0.0554455,0.060198
2,AAP,166.80,N/A,24.27,0.463366,3.0000,0.418812,1.1100,0.171287,11.203493,0.277228,2.508853,0.128713,0.291881
3,AAPL,128.30,N/A,40.57,0.744554,34.8900,0.960396,7.9500,0.843564,27.462022,0.792079,20.229104,0.942574,0.856634
4,ABBV,106.25,N/A,23.89,0.453465,12.6000,0.855446,4.6700,0.676238,15.968442,0.493069,9.340323,0.645545,0.624752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.11,N/A,33.74,0.671287,-4.0000,0.0415842,6.1000,0.767327,24.008699,0.742574,10.403930,0.69505,0.583564
501,ZBH,149.86,N/A,-204.78,0.019802,2.7000,0.381188,4.5000,0.618812,26.457060,0.782178,7.714812,0.49703,0.459802
502,ZBRA,389.14,N/A,45.09,0.780198,10.6200,0.829703,4.7300,0.682178,29.473261,0.825743,11.157335,0.712871,0.766139
503,ZION,43.89,N/A,17.93,0.347525,0.9909,0.0970297,2.2900,0.385149,10.189365,0.225743,2.600916,0.136634,0.238416


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [105]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [106]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.13,N/A,4.94,0.156436,0.4212,0.049505,0.3879,0.0277228,2.500864,0.0336634,0.369976,0.0039604,0.0542574
1,AIG,39.20,N/A,-6.99,0.10495,0.5285,0.0574257,0.7439,0.0673267,4.485190,0.0415842,0.735023,0.0158416,0.0574257
2,AAL,17.20,N/A,-1.24,0.140594,-1.8600,0.0455446,0.4187,0.0316832,-6.221289,0.0277228,1.417907,0.0554455,0.060198
3,HFC,26.79,N/A,-26.80,0.0673267,0.7780,0.0772277,0.2990,0.0138614,6.121347,0.0594059,2.227911,0.116832,0.0669307
4,MET,48.19,N/A,7.63,0.164356,0.5927,0.0613861,0.6546,0.0574257,4.355191,0.039604,0.640350,0.0118812,0.0669307
5,F,9.04,N/A,-17.17,0.0811881,1.1900,0.131683,0.2748,0.0118812,4.805890,0.0475248,2.082945,0.10495,0.0754455
6,HIG,46.36,N/A,10.02,0.190099,0.9529,0.0910891,0.8134,0.0811881,3.852895,0.0376238,0.777520,0.019802,0.0839604
7,ALL,107.41,N/A,7.49,0.162376,1.3200,0.164356,0.7484,0.0693069,2.331697,0.0316832,0.714330,0.0138614,0.0883168
8,BA,236.39,N/A,-28.71,0.0653465,-11.2700,0.029703,2.1000,0.354455,-30.673786,0.0138614,-218.224474,0.0019802,0.0930693
9,KSS,40.91,N/A,-57.00,0.049505,1.2800,0.155446,0.3607,0.0257426,9.575460,0.19604,1.435163,0.0574257,0.0968317


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [107]:
portfolio_input()

Enter the value of your portfolio:10000000


In [108]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/Users/parikshitdubey/.conda/envs/Maps_Hashing/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.13,8646,4.94,0.156436,0.4212,0.049505,0.3879,0.0277228,2.500864,0.0336634,0.369976,0.0039604,0.0542574
1,AIG,39.20,5102,-6.99,0.10495,0.5285,0.0574257,0.7439,0.0673267,4.485190,0.0415842,0.735023,0.0158416,0.0574257
2,AAL,17.20,11627,-1.24,0.140594,-1.8600,0.0455446,0.4187,0.0316832,-6.221289,0.0277228,1.417907,0.0554455,0.060198
3,HFC,26.79,7465,-26.80,0.0673267,0.7780,0.0772277,0.2990,0.0138614,6.121347,0.0594059,2.227911,0.116832,0.0669307
4,MET,48.19,4150,7.63,0.164356,0.5927,0.0613861,0.6546,0.0574257,4.355191,0.039604,0.640350,0.0118812,0.0669307
5,F,9.04,22123,-17.17,0.0811881,1.1900,0.131683,0.2748,0.0118812,4.805890,0.0475248,2.082945,0.10495,0.0754455
6,HIG,46.36,4314,10.02,0.190099,0.9529,0.0910891,0.8134,0.0811881,3.852895,0.0376238,0.777520,0.019802,0.0839604
7,ALL,107.41,1862,7.49,0.162376,1.3200,0.164356,0.7484,0.0693069,2.331697,0.0316832,0.714330,0.0138614,0.0883168
8,BA,236.39,846,-28.71,0.0653465,-11.2700,0.029703,2.1000,0.354455,-30.673786,0.0138614,-218.224474,0.0019802,0.0930693
9,KSS,40.91,4888,-57.00,0.049505,1.2800,0.155446,0.3607,0.0257426,9.575460,0.19604,1.435163,0.0574257,0.0968317


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [125]:
writer = pd.ExcelWriter('ValueStrategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [126]:
background_color = '#fae0df'
font_color = '#495464'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [127]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
    
    

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [128]:
writer.save()